In [2]:
from itertools import repeat

import iris
from iris.coords import AuxCoord
from iris.cube import Cube, CubeList


def make_scalar_coords(name, points):
    return [AuxCoord(point, long_name=name) for point in points]


def make_scalar_cubes(**kwargs):
    # Make a list of scalar cubes with coordinates based on the keyword arguments
    all_scalar_coords = [make_scalar_coords(name, points) 
                         for name, points in kwargs.iteritems()]
    
    cubes = [Cube(0, aux_coords_and_dims=zip(scalar_coords, repeat(None)))
             for scalar_coords in zip(*all_scalar_coords)]
    
    return CubeList(cubes)

a = [1, 1, 1, 2, 2, 2]
b = [1, 2, 3, 1, 2, 3]
c = [1, 2, 3, 4, 5, 6]

cubes = make_scalar_cubes(a=a, b=b, c=c)
print cubes.merge_cube().summary()

unknown / (unknown)                 (a: 2; b: 3)
     Dimension coordinates:
          a                           x     -
          b                           -     x
     Auxiliary coordinates:
          c                           x     x


In [3]:
a = [1,2,1,2]
c = [1,1,2,2]
b = [1,2,3,4]
cubes = make_scalar_cubes(a=a, b=b, c=c)
print cubes.merge_cube().summary()

unknown / (unknown)                 (a: 2; c: 2)
     Dimension coordinates:
          a                           x     -
          c                           -     x
     Auxiliary coordinates:
          b                           x     x


In [4]:
a = [1, 1, 2, 2, 3, 3]
b = [1, 2, 1, 2, 1, 2]
c = [1, 2, 3, 1, 2, 3]
d = [1, 1, 1, 2, 2, 2]

cubes = make_scalar_cubes(a=a, b=b, c=c, d=d)
print cubes.merge_cube().summary()


ValueError: sub-array must be subclass of Array

In [5]:
a, b, c, d = \
    [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
     [0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2],
     [0, 0, 1, 1, 2, 2, 1, 1, 2, 2, 3, 3],
     [2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3]]
    
cubes = make_scalar_cubes(a=a, b=b, c=c, d=d)
print cubes.merge_cube().summary()  

unknown / (unknown)                 (a: 2; b: 3; d: 2)
     Dimension coordinates:
          a                           x     -     -
          b                           -     x     -
          d                           -     -     x
     Auxiliary coordinates:
          c                           x     x     -


In [6]:
import numpy as np

pts = \
    [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
     [0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2],
     [0, 0, 1, 1, 2, 2, 1, 1, 2, 2, 3, 3],
     [2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3]]
    
a = pts[0]
u, i, c = np.unique(a, return_inverse=True, return_counts=True)

print u
print i
print c

np.where(i == 0)

[0 1]
[0 0 0 0 0 0 1 1 1 1 1 1]
[6 6]


(array([0, 1, 2, 3, 4, 5]),)

In [3]:
from collections import deque

import numpy as np

DEBUG_LEVEL = 0
ind = 0


def DEBUG(s, shift=0, level=1):
    global ind
    indent = ' ' * ind
    if level <= DEBUG_LEVEL:
        print indent + str(s)
        
    ind += shift


def _all_same(a):
    a = np.asarray(a)
    return np.all(a[0] ==  a)
    
    
def element_indices(a):
    """
    [10, 20, 10, 30, 20, 30] 
    
    -> { 10: [0, 2], 20: [1, 4], 30: [3, 5]}
    
    [15, 25, 25, 15, 15, 25] 
    
    -> { 15: [0, 3, 4], 25: [1, 2, 5]}
    
    """
    indices = {}
    for i, e in enumerate(a):
        if e in indices:
            indices[e].append(i)
        else:
            indices[e] = deque([i])
    return indices
    
    
def get_new_dims_candidates(scalar_values, rows=None):
    """
    Args:
    
    * scalar_values: list, N*M
    * rows: list, N. The row indices in the original array, for determinig dim_coords indices
    
    Returns:
    
    * candidate_shapes: list, K
    * candidate_dim_coords: list, K
    
    [[1, 2]]
    
     --> [(2,)], [[0]]
    
    
    [[1, 1, 2, 2, 3, 3],
     [1, 2, 1, 2, 1, 2],
     [1, 2, 1, 2, 2, 1]]
     
     --> [(3, 2), (3, 2)],
         [[0, 1], [0, 2]]

    
    [[1, 2, 3, 4],
     [1, 2, 1, 2],
     [1, 1, 2, 2],
     [1, 1, 2, 2]]
     
     --> [(2, 2), (2, 2)],
         [[1, 2], [1, 3]]
    
    
    [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
     [0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2],
     [0, 0, 1, 1, 2, 2, 1, 1, 2, 2, 3, 3],
     [2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3]]

     --> [(2, 3, 2)], 
         [[0, 1, 3]]
        
        
    [[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2],
     [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
     [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6],
     [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]])
     
     --> [(2, 6), (3, 4)],
         [[0, 2], [1, 3]]
         
         
    [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
     [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2],
     [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6],
     [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
     [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]]
     
     --> [(2, 2, 6), (2, 3, 4)],
         [[0, 1, 2], [0, 3, 4]]
     
    
    """
    if rows is None:
        rows = np.arange(len(scalar_values))
    scalar_values = np.asarray(scalar_values)
        
    ncoords, nvalues = scalar_values.shape
    if rows is None:
        rows = np.arange(ncoords)
        
    if nvalues < 2:
        return []

    # For each row, get a dictionary of the indices for each distinct element.
    # E.g. the element indices for [10, 20, 10, 30, 30] are:
    # {10: [0, 2], 
    #  20: [1],
    #  30: [3, 4]}
    eis = [element_indices(a) for a in scalar_values]
    
    candidate_shapes = []
    candidate_dim_coords = []
    for ei in eis:
        row = rows[0]
        DEBUG('Trying row {}'.format(row))
        indices = ei.values()
        counts = np.array([len(v) for v in indices])
        # To be a dim coord, the number of occurences of each element must be the same.
        if np.all(counts == counts[0]):
            count = counts[0]
            dim_len = nvalues / count
            DEBUG('It is a potential candidate dimension of length {}'.format(dim_len))
            DEBUG('    {}'.format(indices), level=2)
            if count == 1:
                # If there is only one occurence of each element then this row can form a dimension
                # with no additional dimensions.
                DEBUG('Its values are all distinct')
                new_shape = (dim_len, count) if count > 1 else (dim_len,)
                DEBUG('Appending shape: {}'.format(new_shape))
                DEBUG('Appending row: {}'.format(row))
                candidate_shapes.append(new_shape)
                candidate_dim_coords.append([row])
            else:
                # Otherwise, we need to find a row or rows that can form the remaining dimensions.
                DEBUG('Its values are not distinct.')
                DEBUG('Finding independent coords')
                for a in scalar_values:
                    DEBUG([list(set(a[idx])) for idx in indices], level=2)
                independents = []
                # Find which rows are 'independent' of this row. That is, for each of ther other rows check that:
                # - The sets of values at the indices of each element in the current row are all the same.
                #   E.g: current row [10, 10, 10, 11, 11, 11]
                #        row1        [20, 21, 22, 20, 21, 23]
                #        row2        [30, 31, 31, 31, 30, 31]
                #        row3        [40, 41, 42, 41, 40, 42]
                #   The indices are:
                #                    {10: [0, 1, 2], 11: [3, 4, 5]}
                #   The sets of values for each row at these indices are:
                #        row1        [20, 21, 22], [20, 21, 23]
                #        row2        [30, 31], [30, 31]
                #        row3        [40, 41, 42], [40, 41, 42]
                #   The set of values at each list of indices for row2 and row3 are all the same.
                # - The number of values for each element at the indices of each element in the current row 
                #   are all equal.
                #   E.g: The set of values for each element at each list of indices for row2 and row3 are:
                #        row2        {30: [0], 31: [1, 2] }, {30: [1], 31: [0, 2]}
                #        row3        {40: [0], 41: [1], 42: [2]}, {40: [1], 41: [0], 42: [2]}
                #   So the number of values for each element are:
                #        row2        {30: 1, 31: 2}, {30: 1, 31: 2}
                #        row3        {40: 1, 41: 1, 42: 1}, {40: 1, 41: 1, 42: 1}
                #   Therefore the number of values for each element at each list of indices are the same for row3.
                for i, a in enumerate(scalar_values[1:]):
                    # Check whether the set of values at each list of indices are all the same for this row.
                    if _all_same([set(a[j]) for j in indices]):
                        sub_eis = [element_indices(a[idx]) for idx in indices]
                        # Check whether the number of values for each element at each list of indices are all the same 
                        # for this row.
                        if _all_same([len(j) for sub_ei in sub_eis
                                      for j in sub_ei.values()]):
                            # This row is indpendent of the current row.
                            independents.append(i + 1)

                # Recurse with only the rows identified in the step above, passing only the values at one of the lists of
                # indices.
                # E.g. current row [10, 10, 10, 11, 11, 11]
                #      row1        [20, 21, 22, 20, 21, 22]
                #      row2        [30, 31, 32, 30, 34, 35]
                #      row3        [41, 40, 42, 40, 41, 42]
                # We recurse with:
                #      row1        [20, 21, 22]
                #      row3        [41, 40, 42]
                # Then append the current row index and its length as a dimension coordinate to each element in the list
                # of dimension coordinates and shapes identified at the next level of recursion respectively.
                # E.g. current row length as a dim coord == 2. The next level of recursion identifies both row1 and row3
                # as candidate dimension coordinates, so the return values are:
                #      sub_shapes = [(3,), (3,)],
                #      sub_dim_coords = [[1], [3]]
                # After appending the current row index and length as a dim_coord we have:
                #      new_shapes = [(2, 3), (2, 3)]
                #      new_dim_coords = [[0, 1], [0, 3]]
                if independents:
                    # Only look at one subset of the values for the independent rows.
                    sub_values = scalar_values[independents][..., indices[0]]
                    DEBUG('There are potential extra dimensions at rows {}'.format([ rows[i] for i in independents]))
                    DEBUG('Recursing with new array:')
                    DEBUG(sub_values, shift=4)
                    sub_shapes, sub_dim_coords = get_new_dims_candidates(sub_values, rows[independents])
                    DEBUG('Extending shapes: {}'.format([(dim_len,) + shape for shape in sub_shapes]))
                    DEBUG('Extending rows: {}'.format([[row] + dim_coords for dim_coords in sub_dim_coords]))
                    # Extend each sub- dimension coord list and sub-shape with the current row index and length as a
                    # dimension coordinate.
                    candidate_shapes.extend([(dim_len,) + shape for shape in sub_shapes])
                    candidate_dim_coords.extend([[row] + dim_coords for dim_coords in sub_dim_coords])
                else:
                    DEBUG('There are no possible extra dimensions. Not adding this dimension.')
        # Eliminate this row from the search.
        scalar_values = scalar_values[1:]
        rows = rows[1:]

    DEBUG('Shapes found at this level: {}'.format(candidate_shapes))
    DEBUG('Dim coords found at this level: {}'.format(candidate_dim_coords), shift=-4)
    return candidate_shapes, candidate_dim_coords


#print get_new_dims_candidates([[1,1,1,2,2,2],
#                               [1,2,1,1,2,2]])

print get_new_dims_candidates([[1,1,2,2],
                               [3,3,4,4],
                               [5,6,5,6]])

#print get_new_dims_candidates([[1,1,1,2,2,2],
#                               [1,2,3,1,2,3],
#                               [4,5,6,4,5,6]])
    
#print get_new_dims_candidates([[1,2]])
#print get_new_dims_candidates([[1,2,1,2],
#                               [1,1,2,2]])
#print get_new_dims_candidates([[1,2,1,2],[1,2,3,4]])
#print get_new_dims_candidates(
#    [[1, 1, 2, 2, 3, 3],
#     [5, 6, 5, 6, 5, 6],
#     [7, 8, 7, 8, 8, 7]])

#print get_new_dims_candidates(
#    [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
#     [0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2],
#     [0, 0, 1, 1, 2, 2, 1, 1, 2, 2, 3, 3],
#     [2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3]])

#print get_new_dims_candidates([[1,1,1,1,1,1,2,2,2,2,2,2],
#                               [1,1,1,1,2,2,2,2,3,3,3,3],
#                               [1,2,3,4,5,6,1,2,3,4,5,6],
#                               [1,2,3,4,1,2,3,4,1,2,3,4]])


#print get_new_dims_candidates(
#    [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
#     [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2],
#     [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6],
#     [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
#     [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]])

([(2, 2), (2, 2)], [[0, 2], [1, 2]])


In [61]:
from iris._merge import _Skeleton
from iris.coords import Cell, DimCoord, AuxCoord

N = 12
shape = (2,3)

datas = [np.tile(i % 10, shape) for i in range(1, N+1)]


#a = np.arange(N) % 3 + 10
#b = np.arange(N) / 3 + 20

names = ['x', 'a', 'b', 'c']

x =[1,2,2,2,1,1,2,1,2,1,2,1]
a =[4,4,4,4,3,4,3,3,3,4,3,3]
b =[7,6,5,7,5,5,6,7,7,6,5,6]
c =[6,5,4,6,1,4,2,3,3,5,1,2]

skeletons = [_Skeleton([Cell(xi), Cell(ai), Cell(bi), Cell(ci)], di) for xi, ai, bi, ci, di in zip(x, a, b, c, datas)]

scalar_values = np.empty((4, 12), dtype=object)
scalar_values[:] = list(zip(*[[xi, ai, bi, ci] for xi, ai, bi,ci  in (sk.scalar_values for sk in skeletons)]))

datas = np.empty((12,), dtype=object)
datas[:] = [sk.data for sk in skeletons]

print datas.shape

# Get candidate shapes and dim coords
candidate_shapes, candidate_dim_coords = get_new_dims_candidates(scalar_values)


print 'CDC=', candidate_dim_coords
# Pick the first for now
shape = candidate_shapes[0]
dim_coords = candidate_dim_coords[0]


"""
a =[1,2,3,6,5,6,3,4,2,5,4,1]
b =[1,1,1,2,1,1,2,2,2,2,1,2]
d =[1,2,3,4,5,6,7,8,9,0,1,2]

sort by a ->

a =[1,1,2,2,3,3,4,4,5,5,6,6]
b =[1,2,1,2,1,2,2,1,1,2,2,1]
d =[1,2,2,9,3,7,8,1,5,0,4,6]

sort by b ->

a =[1,1,2,2,3,3,4,4,5,5,6,6]
b =[1,2,1,2,1,2,1,2,1,2,1,2]
d =[1,2,2,9,3,7,1,8,5,0,6,4]


"""

def cells_points(cells):
    ret = np.empty_like(cells, dtype=float)
    ret.flat = [cell.point for cell in cells.flat]
    return ret

def cells_bounds(cells):
    if cells.flat[0].bound is not None:
        ret = np.empty(cells.shape + (2,), dtype=float)
        ret.flat = np.array([cell.bound for cell in cells.flat])
        return ret
    else:
        return None


print shape
print 'DC=', dim_coords
print

data_stack = datas

print '********'
print x
print a
print b
print c
print
print [d.flat[0] for d in datas]
print '********'


def order_and_reshape(scalar_values, data_stack, dim_coords, shape):
    final_order = np.arange(scalar_values.shape[1])
    for dim_coord in dim_coords[::-1]:
        order = np.argsort(scalar_values[dim_coord])
        scalar_values = scalar_values[:, order]
        final_order = final_order[order]

    data_stack = data_stack[final_order]
    
    scalar_values.shape = (-1,) + shape
    data_stack.shape = shape
    return scalar_values, data_stack
    

scalar_values, data_stack = order_and_reshape(scalar_values, data_stack, dim_coords, shape)

dim_coord_dims = np.repeat(-1, len(scalar_values))
dim_coord_dims[dim_coords] = range(len(dim_coords))

def construct_coordinates(scalar_values, dim_coords_dims, ndims):
    dim_coords_and_dims = []
    aux_coords_and_dims = []
    for i in range(len(scalar_values)):
        if dim_coord_dims[i] != -1:
            value_slice = [0] * (ndims + 1)
            value_slice[0] = i
            value_slice[dim_coord_dims[i] + 1] = slice(None)
            value_slice = tuple(value_slice)
            points = cells_points(scalar_values[value_slice])
            bounds = cells_bounds(scalar_values[value_slice])
            dim_coords_and_dims.append((DimCoord(points, bounds=bounds, long_name=names[i]), dim_coord_dims[i]))
        else:
            value_slice = [i]
            coord_dims = []
            for j in range(len(dim_coords)):
                dim_j = dim_coords[j]
                dim_j_dim = dim_coord_dims[dim_j]
                slc = [slice(None)] * (ndims + 1)
                slc[0] = i
                slc[dim_j_dim + 1] = 0
                #slc = (i,) + (slice(None),)*dim_j_dim + (0,) + (slice(None),)*(len(dim_coords) - dim_j_dim - 1)
                if np.all(scalar_values[slc] == scalar_values[i]):
                    value_slice.append(0)
                else:
                    value_slice.append(slice(None))
                    coord_dims.append(dim_j_dim)
            value_slice = tuple(value_slice)
            coord_dims = tuple(coord_dims)
            points = cells_points(scalar_values[value_slice])
            bounds = cells_bounds(scalar_values[value_slice])
            aux_coords_and_dims.append((AuxCoord(points, bounds=bounds, long_name=names[i]), coord_dims))
    
    return dim_coords_and_dims, aux_coords_and_dims
        
        
dim_coords_and_dims, aux_coords_and_dims = construct_coordinates(scalar_values, dim_coord_dims, len(dim_coords))  
            

for coord in dim_coords_and_dims:
    print coord
    
for coord in aux_coords_and_dims:
    print coord  
    
shp = data_stack.flat[0].shape
cube = Cube(np.stack(data_stack.ravel()).reshape(stk_shp + shp), dim_coords_and_dims=dim_coords_and_dims, aux_coords_and_dims=aux_coords_and_dims)

print cube



(12,)
CDC= [[0, 1, 2], [0, 3]]
(2, 2, 3)
DC= [0, 1, 2]

********
[1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 1]
[4, 4, 4, 4, 3, 4, 3, 3, 3, 4, 3, 3]
[7, 6, 5, 7, 5, 5, 6, 7, 7, 6, 5, 6]
[6, 5, 4, 6, 1, 4, 2, 3, 3, 5, 1, 2]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2]
********
(DimCoord(array([ 1.,  2.]), standard_name=None, units=Unit('1'), long_name='x'), 0)
(DimCoord(array([ 3.,  4.]), standard_name=None, units=Unit('1'), long_name='a'), 1)
(DimCoord(array([ 5.,  6.,  7.]), standard_name=None, units=Unit('1'), long_name='b'), 2)
(AuxCoord(array([[ 1.,  2.,  3.],
       [ 4.,  5.,  6.]]), standard_name=None, units=Unit('1'), long_name='c'), (1, 2))
unknown / (unknown)                 (x: 2; a: 2; b: 3; -- : 2; -- : 3)
     Dimension coordinates:
          x                           x     -     -       -       -
          a                           -     x     -       -       -
          b                           -     -     x       -       -
     Auxiliary coordinates:
          c                 

/net/data/users/dkirkham/miniconda2/envs/iris-dev/lib/python2.7/site-packages/ipykernel_launcher.py:131: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [49]:
a = np.empty((2,2), dtype=object)
a[:] = [[Cell(0, None), Cell(1, None)], [Cell(2, None), Cell(3, None)]]



print a

[[Cell(point=0, bound=None) Cell(point=1, bound=None)]
 [Cell(point=2, bound=None) Cell(point=3, bound=None)]]


In [3]:
l = [[1,1,2,1,3,4,3],
     [5,5,3,5,4,7,4]]

[element_indices(a) for a in l]

[{1: deque([0, 1, 3]), 2: deque([2]), 3: deque([4, 6]), 4: deque([5])},
 {3: deque([2]), 4: deque([4, 6]), 5: deque([0, 1, 3]), 7: deque([5])}]

In [15]:
d = AuxCoord([1,1,1,2,2,2], long_name='d')
e = AuxCoord([2,3,1,4,5,6], long_name='e')
cube = Cube([1,2,3,4,5,6], aux_coords_and_dims=[(d,0), (e,0)])

se = e.copy(points=sorted(e.points))
cube.subset(e)

print cube

agg = cube.aggregated_by('d', iris.analysis.MEAN)

print agg.coord('e')

unknown / (unknown)                 (-- : 6)
     Auxiliary coordinates:
          d                             x
          e                             x
AuxCoord(array([ 1.5,  5. ]), bounds=array([[2, 1],
       [4, 6]]), standard_name=None, units=Unit('1'), long_name='e')
